In [13]:
import pandas as pd 
import numpy as np
import pandas as pd
import numpy as np 
import spacy 
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.tokenize import  word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

C:\Users\Master\anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [14]:
df = pd.read_csv('file1.csv',encoding='utf-8')


In [15]:
df.drop_duplicates(subset=['Context'],inplace=True)
df.head()

Unnamed: 0                                            Context  \
0            0  I'm going through some things with my feelings...   
23          23  I have so many issues to address. I have a his...   
70          70  I have been feeling more and more down for ove...   
72          72  I’m facing severe depression and anxiety and I...   
81          81  How can I get to a place where I can be conten...   

                                             Response  
0   If everyone thinks you're worthless, then mayb...  
23  Let me start by saying there are never too man...  
70  Answers about our inner lives are most success...  
72  Have you used meditation or hypnosis? Relaxing...  
81  Your question is a fascinating one!As humans w...

In [16]:
df.dropna(inplace=True)


In [17]:
df.shape

(1300, 3)

In [18]:
def process(text):
    text = text.lower()
    
    doc = nlp(text)
    
    filtered_tokens = []
    
    for token in doc :
        # Remove stop words & punc
        
        if token.is_punct or token.is_stop:
            continue
            
        filtered_tokens.append(token.lemma_)
    return " ".join(filtered_tokens)



In [19]:
df['Context'] = df['Context'].apply(process)
df['Response'] = df['Response'].apply(process)
df.head()

Unnamed: 0                                            Context  \
0            0  go thing feeling barely sleep think worthless ...   
23          23  issue address history sexual abuse breast canc...   
70          70  feel month start have trouble sleep panic atta...   
72          72  face severe depression anxiety feel like go lo...   
81          81                              place content day day   

                                             Response  
0   think worthless maybe need find new people han...  
23  let start say concern bring counselling fact p...  
70  answer inner life successfully reach sense fee...  
72  meditation hypnosis relax mind connect true se...  
81  question fascinating one!as human ability refl...

In [20]:

df.head()

Unnamed: 0                                            Context  \
0            0  go thing feeling barely sleep think worthless ...   
23          23  issue address history sexual abuse breast canc...   
70          70  feel month start have trouble sleep panic atta...   
72          72  face severe depression anxiety feel like go lo...   
81          81                              place content day day   

                                             Response  
0   think worthless maybe need find new people han...  
23  let start say concern bring counselling fact p...  
70  answer inner life successfully reach sense fee...  
72  meditation hypnosis relax mind connect true se...  
81  question fascinating one!as human ability refl...

In [21]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = df["Response"]
y = encoder.fit_transform(y)
def preprocess_text_data(df, text_column, max_sequence_length=None):
    """
    Preprocesses text data in a DataFrame for input to an RNN model.
    
    Args:
    - df: DataFrame containing the text data.
    - text_column: Name of the column containing the text data.
    - max_sequence_length: Maximum sequence length for padding sequences.
      If None, the maximum sequence length is determined dynamically from the data.
    
    Returns:
    - padded_sequences: Padded sequences of word indices.
    - word_index: Word index mapping from words to indices.
    """
    # Extract text data from DataFrame
    texts = df[text_column].tolist()
    
    # Initialize a Tokenizer
    tokenizer = Tokenizer()
    # Fit the tokenizer on the text data
    tokenizer.fit_on_texts(texts)
    # Convert the text data to sequences of word indices
    sequences = tokenizer.texts_to_sequences(texts)
    
    # Determine maximum sequence length if not provided
    if max_sequence_length is None:
        max_sequence_length = max(len(seq) for seq in sequences)
    
    # Pad sequences to ensure uniform length
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
    
    # Get the word index mapping
    word_index = tokenizer.word_index
    
    return padded_sequences, word_index

# Example usage:
# Assuming df is your DataFrame and 'text_column' is the name of the column containing text data
padded_sequences, word_index = preprocess_text_data(df, 'Context')
print("Padded Sequences:", padded_sequences)
print("Word Index:", word_index)


Padded Sequences: [[  11   17   49 ...    0    0    0]
 [  48  693  694 ...    0    0    0]
 [   1   29   26 ...    0    0    0]
 ...
 [ 547   82  196 ...    0    0    0]
 [ 630 1049 1765 ...    0    0    0]
 [1765 2623    0 ...    0    0    0]]
Word Index: {'feel': 1, 'want': 2, 'like': 3, 'know': 4, 'time': 5, 'year': 6, 'tell': 7, 'think': 8, 'love': 9, 'say': 10, 'go': 11, 'help': 12, 'talk': 13, 'relationship': 14, '\r': 15, 'boyfriend': 16, 'thing': 17, 'friend': 18, 'try': 19, 'get': 20, 'life': 21, 'find': 22, 'work': 23, 'need': 24, 'family': 25, 'start': 26, 'people': 27, 'day': 28, 'month': 29, 'lot': 30, 'have': 31, 'stop': 32, 'make': 33, 'leave': 34, 'child': 35, 'good': 36, 'sex': 37, 'anxiety': 38, 'come': 39, 'ago': 40, 'husband': 41, 'past': 42, 'live': 43, 'break': 44, 'ask': 45, 'problem': 46, 'depression': 47, 'issue': 48, 'feeling': 49, 'wrong': 50, 'ex': 51, 'school': 52, 'lose': 53, 'mom': 54, 'parent': 55, 'bad': 56, 'away': 57, 'girl': 58, 'way': 59, 'date': 6

In [22]:
vocab_size = len(word_index) + 1  # Add 1 for the padding token (index 0)

# Set embedding_dim
embedding_dim = 100  # Example value, adjust as needed

# Set max_sequence_length based on the maximum sequence length in your data
max_sequence_length = padded_sequences.shape[1]  # Assuming padded_sequences is your preprocessed data

# Print the obtained values
print("Vocabulary Size:", vocab_size)
print("Embedding Dimension:", embedding_dim)
print("Max Sequence Length:", max_sequence_length)

Vocabulary Size: 2624
Embedding Dimension: 100
Max Sequence Length: 178


In [23]:
from sklearn.model_selection import train_test_split

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Split the data into training and temporary sets
X_train_val, X_test, y_train_val, y_test = train_test_split(padded_sequences, y, test_size=test_ratio, random_state=42)

# Split the temporary set into validation and test sets
relative_val_ratio = val_ratio / (val_ratio + test_ratio)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=relative_val_ratio, random_state=42)

# Print the shapes of the resulting sets
#print("Training set shape:", X_train.shape, y_train.shape)
#print("Validation set shape:", X_val.shape, y_val.shape)
#print("Test set shape:", X_test.shape, y_test.shape)
def show_shapes(): # can make yours to take inputs; this'll use local variable values
    print("Expected: (num_samples, timesteps, channels)")
    print("Sequences: {}".format(X_train_val.shape))
    print("Targets:   {}".format(y_train_val.shape))   
show_shapes()

Expected: (num_samples, timesteps, channels)
Sequences: (1105, 178)
Targets:   (1105,)


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD

# Define SGD optimizer for Model 1
sgd_1 = SGD(learning_rate=0.01, momentum=0.9)

# Define model architecture for Model 1
model_1 = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)),
    Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)),
    Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)),
    Dense(units=64, activation='relu'),
    Dropout(0.5),
    Dense(units=vocab_size, activation='softmax')
])

# Compile Model 1 with SGD optimizer
model_1.compile(optimizer=sgd_1, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback for Model 1
early_stopping_1 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train Model 1
history_1 = model_1.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping_1])

# Define SGD optimizer for Model 2
sgd_2 = SGD(learning_rate=0.01, momentum=0.9)

# Define model architecture for Model 2
model_2 = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    Bidirectional(GRU(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)),
    Bidirectional(GRU(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)),
    Bidirectional(GRU(units=128, dropout=0.2, recurrent_dropout=0.2)),
    Dense(units=64, activation='relu'),
    Dropout(0.5),
    Dense(units=vocab_size, activation='softmax')
])

# Compile Model 2 with SGD optimizer
model_2.compile(optimizer=sgd_2, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback for Model 2
early_stopping_2 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train Model 2
history_2 = model_2.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping_2])

# Evaluate both models on the test set
loss_1, accuracy_1 = model_1.evaluate(X_test, y_test)
loss_2, accuracy_2 = model_2.evaluate(X_test, y_test)

print("Model 1 Test Loss:", loss_1)
print("Model 1 Test Accuracy:", accuracy_1)
print("Model 2 Test Loss:", loss_2)
print("Model 2 Test Accuracy:", accuracy_2)


Epoch 1/100


C:\Users\Master\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 7.8085e-04 - loss: 7.8726 - val_accuracy: 0.0018 - val_loss: 7.8724
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.0049 - loss: 7.8715 - val_accuracy: 0.0072 - val_loss: 7.8721
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.0046 - loss: 7.8700 - val_accuracy: 0.0072 - val_loss: 7.8719
Epoch 4/100
2/9 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.0156 - loss: 7.8683

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

